#  ASR Project Demo (CTC-BiLSTM)

Демонстрация работы системы распознавания речи.


## 1.  Установка и Скачивание кода

Этот блок автоматически скачает код из репозитория **Olga57/speech-recognition-ctc**,
установит зависимости и подготовит окружение.

In [ ]:
# 1. Клонируем репозиторий
import os

USER = 'Olga57'
REPO = 'speech-recognition-ctc'

if not os.path.exists(REPO):
    print(f'Скачиваем проект {USER}/{REPO}...')
    !git clone https://github.com/{USER}/{REPO}.git

# 2. Заходим в папку проекта
%cd {REPO}

# 3. Устанавливаем библиотеки
print('Установка зависимостей...')
!pip install -q -r requirements.txt
!pip install -q gdown
!apt-get install -y libsndfile1 ffmpeg > /dev/null
print(' Готово! Окружение настроено.')

## 2.  Быстрый тест (Librispeech)
Запуск модели на валидационных данных (проверка, что веса скачались).

In [ ]:
!python inference.py \
    dataset=librispeech \
    dataset.valid_split=validation.clean \
    dataset.max_valid_items=5 \
    device='cuda' \
    decode='beam' \
    out_dir='predictions_libri'

!head -n 5 predictions_libri/*.txt

## 3.  Тест на вашем датасете (Google Drive)
Вставьте ссылку на ZIP-архив с Google Drive. Внутри должна быть папка `audio/` и `transcriptions/`.

In [ ]:
import gdown
import os

url = input('Ссылка на Google Drive (ZIP): ')

if 'drive.google.com' in url:
    file_id = url.split('/d/')[-1].split('/')[0]
    dl_url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(dl_url, 'custom.zip', quiet=False)
    !unzip -q -o custom.zip -d custom_data
    
    # Ищем корень датасета
    ds_root = 'custom_data'
    for r, d, f in os.walk('custom_data'):
        if 'audio' in d: ds_root = r

    print(f'Датасет распакован в: {ds_root}')
    
    !python inference.py dataset=custom dataset.custom_root="{ds_root}" device='cuda' decode='beam' out_dir='preds_custom'
    !python calc_metrics.py --ref_dir "{ds_root}/transcriptions" --hyp_dir "preds_custom"
else:
    print('Это не ссылка на Google Drive')